<a href="https://colab.research.google.com/github/YorkWestenhaver/RL/blob/main/Nightshade_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog uses FinRL to reproduce the paper: Practical Deep Reinforcement Learning Approach for Stock Trading, Workshop on Challenges and Opportunities for AI in Financial Services, NeurIPS 2018.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-x_7zdlre
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-x_7zdlre
  Created wheel for finrl: filename=finrl-0.3.0-cp37-none-any.whl size=38749 sha256=3ec2448b2a5b0b6b6220254c661cd6bc7ed4a1b939f2a5bf39db3be7c19638ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-lmr0zoco/wheels/9c/19/bf/c644def96612df1ad42c94d5304966797eaa3221dffc5efe0b
Successfully built finrl



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

In [ ]:
!pip install pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Create Folders

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
print(os.path)

<module 'posixpath' from '/usr/lib/python3.7/posixpath.py'>


<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [ ]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [ ]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [ ]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [ ]:
config.DOW_30_TICKER.append('BTC-USD')

In [ ]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-04-28',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df.shape

(93000, 8)

In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.698216,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.592946,40980600,CSCO,4


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True).tail(60)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
92940,2021-04-26,AAPL,134.830002,135.059998,133.559998,134.720001,66905100.0,0.0,2.579714,140.027122,119.832879,56.382794,95.153929,24.829268,127.375333,127.804278,19.574758
92941,2021-04-26,AXP,146.800003,151.119995,145.559998,150.339996,4821100.0,0.0,1.757181,151.161701,141.570634,59.626867,109.328550,4.994538,144.726394,139.161916,19.574758
92942,2021-04-26,BA,240.240005,244.649994,240.100006,241.440002,10599600.0,0.0,-1.225832,263.554900,233.187099,51.957661,-80.368990,5.198037,249.583666,234.195166,19.574758
92943,2021-04-26,CAT,230.570007,232.850006,229.960007,230.559998,1864800.0,0.0,2.158297,233.460004,227.134205,58.518677,45.649095,4.874326,228.726497,217.720874,19.574758
92944,2021-04-26,CSCO,51.650002,51.770000,51.500000,51.639999,13778200.0,0.0,0.710775,52.724795,51.012162,59.233313,29.994468,10.230529,51.104512,48.669856,19.574758
92945,2021-04-26,CVX,101.269997,102.669998,101.260002,101.519997,5639700.0,0.0,-0.527991,106.662071,100.099928,51.427340,-93.644615,11.581545,104.093333,100.882747,19.574758
92946,2021-04-26,DD,76.980003,78.000000,76.769997,76.930000,1870800.0,0.0,0.146171,78.651846,74.877152,52.836886,27.025334,5.107800,76.924999,74.988558,19.574758
92947,2021-04-26,DIS,183.960007,185.600006,183.259995,184.270004,6178300.0,0.0,-1.426179,190.341448,181.746553,49.942275,-80.235388,0.493970,187.656000,188.007167,19.574758
92948,2021-04-26,GS,341.940002,345.899994,340.890015,343.519989,2171500.0,0.0,2.835859,344.748028,320.368973,59.493886,99.041536,19.893543,334.459000,324.099938,19.574758
92949,2021-04-26,HD,323.679993,324.000000,319.320007,319.989990,2848500.0,0.0,9.352921,332.857190,302.088809,63.621069,66.388471,30.292541,307.834333,287.936920,19.574758


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2018-12-31
## Trade data split: 2019-01-01 to 2020-09-30

In [ ]:
train = data_split(processed_full, '2009-01-01','2019-01-01')
trade = data_split(processed_full, '2019-01-01','2021-04-28')
print(len(train))
print(len(trade))

75480
17520


In [ ]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.791740,746015200.0,4.0,0.0,3.017278,2.684025,100.0,66.666667,100.0,2.791740,2.791740,0.0
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.698216,10955700.0,4.0,0.0,3.017278,2.684025,100.0,66.666667,100.0,15.698216,15.698216,0.0
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.0,3.017278,2.684025,100.0,66.666667,100.0,33.941101,33.941101,0.0
0,2009-01-02,CAT,44.910000,46.980000,44.709999,32.830360,7117200.0,4.0,0.0,3.017278,2.684025,100.0,66.666667,100.0,32.830360,32.830360,0.0
0,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.592946,40980600.0,4.0,0.0,3.017278,2.684025,100.0,66.666667,100.0,12.592946,12.592946,0.0


In [ ]:
trade.head(100)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.505024,148158800.0,2.0,-2.016889,44.505522,35.444587,37.867340,-91.571542,42.250808,41.225720,46.488189,51.409024
0,2019-01-02,AXP,93.910004,96.269997,93.769997,92.042946,4175400.0,2.0,-3.403805,109.907410,84.174485,41.204998,-97.759092,26.709417,99.719922,100.286694,51.409024
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-5.550592,339.116375,287.252015,47.010000,-21.712382,13.611972,314.427160,331.956148,51.409024
0,2019-01-02,CAT,124.029999,127.879997,123.000000,118.137177,4783200.0,2.0,-0.680049,125.145761,109.015061,48.229091,-5.147002,0.873482,117.640500,119.834356,51.409024
0,2019-01-02,CSCO,42.279999,43.200001,42.209999,39.772106,23833500.0,2.0,-0.945558,46.120801,36.852423,44.872560,-87.538213,29.529377,41.863034,42.137596,51.409024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2019-01-07,CVX,111.059998,112.930000,109.919998,100.669258,5712700.0,0.0,-1.248962,106.501632,91.499203,48.542900,-16.233493,3.407887,101.006749,102.153458,20.674216
3,2019-01-07,DD,77.995178,79.005707,77.041595,74.424515,7724397.0,0.0,-0.700248,75.822389,67.855257,47.411612,15.539746,14.912548,73.725677,75.443788,20.674216
3,2019-01-07,DIS,109.910004,111.400002,109.300003,109.225136,6714700.0,0.0,-1.118034,114.051519,101.241347,49.389837,-5.994116,5.847643,109.104763,111.115668,20.674216
3,2019-01-07,GS,175.229996,177.830002,172.270004,167.749557,3152100.0,0.0,-5.436011,174.578136,149.950797,41.955592,-13.232762,0.498431,168.842192,187.277307,20.674216


In [ ]:
config.TECHNICAL_INDICATORS_LIST

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 301


In [ ]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

#originally the total_timesteps was 100000

Logging to tensorboard_log/a2c/a2c_1
------------------------------------
| time/                 |          |
|    fps                | 81       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -44.1    |
|    explained_variance | -0.146   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -110     |
|    std                | 1        |
|    value_loss         | 6.91     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 82       |
|    iterations         | 200      |
|    time_elapsed       | 12       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -44.1    |
|    explained_variance | 0.0169   |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

Logging to tensorboard_log/ddpg/ddpg_1
----------------------------------
| environment/        |          |
|    portfolio_value  | 3.49e+06 |
|    total_cost       | 999      |
|    total_reward     | 2.49e+06 |
|    total_reward_pct | 249      |
|    total_trades     | 35210    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 60       |
|    time_elapsed     | 199      |
|    total timesteps  | 12012    |
| train/              |          |
|    actor_loss       | -4.4     |
|    critic_loss      | 942      |
|    learning_rate    | 0.001    |
|    n_updates        | 9009     |
----------------------------------
day: 3002, episode: 30
begin_total_asset: 1000000.00
end_total_asset: 3490805.93
total_reward: 2490805.93
total_cost: 999.00
total_trades: 35210
Sharpe: 4.523
----------------------------------
| environment/        |          |
|    portfolio_value  | 3.49e+06 |
|    total_cost       | 999      |
|    total_reward     | 2.49e+06

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

Logging to tensorboard_log/ppo/ppo_2
-----------------------------
| time/              |      |
|    fps             | 107  |
|    iterations      | 1    |
|    time_elapsed    | 19   |
|    total_timesteps | 2048 |
-----------------------------


KeyboardInterrupt: ignored

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

Logging to tensorboard_log/td3/td3_2
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 33       |
|    time_elapsed    | 296      |
|    total timesteps | 10064    |
| train/             |          |
|    actor_loss      | 67.9     |
|    critic_loss     | 979      |
|    learning_rate   | 0.001    |
|    n_updates       | 7548     |
---------------------------------
day: 2515, episode: 10
begin_total_asset:1000000.00
end_total_asset:4438572.29
total_reward:3438572.29
total_cost: 1038.05
total_trades: 40290
Sharpe: 1.049
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 30       |
|    time_elapsed    | 669      |
|    total timesteps | 20128    |
| train/             |          |
|    actor_loss      | 54       |
|    critic_loss     | 199      |
|    learning_rate   | 0.001    |
|    n_updates       | 17612    |
------------------------

### Model 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device


In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=80000)

Logging to tensorboard_log/sac/sac_2
----------------------------------
| environment/        |          |
|    portfolio_value  | 2.6e+06  |
|    total_cost       | 1.69e+05 |
|    total_reward     | 1.6e+06  |
|    total_reward_pct | 160      |
|    total_trades     | 65253    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 41       |
|    time_elapsed     | 241      |
|    total timesteps  | 10064    |
| train/              |          |
|    actor_loss       | 1.02e+03 |
|    critic_loss      | 60       |
|    ent_coef         | 0.176    |
|    ent_coef_loss    | -64.9    |
|    learning_rate    | 0.0001   |
|    n_updates        | 9963     |
----------------------------------
----------------------------------
| environment/        |          |
|    portfolio_value  | 4.15e+06 |
|    total_cost       | 1.25e+04 |
|    total_reward     | 3.15e+06 |
|    total_reward_pct | 315      |
|    total_trades     | 44691    |
| time/           

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_turbulence = processed_full[(processed_full.date<'2019-01-01') & (processed_full.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
insample_turbulence.turbulence.describe()

count    3003.000000
mean       36.475309
std        50.771388
min         0.000000
25%         5.908978
50%        25.006747
75%        45.294562
max       648.650094
Name: turbulence, dtype: float64

In [ ]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [ ]:
turbulence_threshold

648.65009432749

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
trade = data_split(processed_full, '2019-01-01','2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 380, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-01,AAPL,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2019-01-01,AXP,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2019-01-01,BA,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2019-01-01,BTC-USD,3746.713379,3850.913818,3707.231201,3843.52002,4.324201e+09,1.0,-54.957248,4293.981012,3174.069671,43.087782,30.190815,4.840104,3691.036963,4474.115735,0.940493
0,2019-01-01,CAT,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [ ]:
df_account_value.shape

(731, 2)

In [ ]:
print(df_actions.to_string())

In [ ]:
df_account_value.tail()

,date,account_value
726,2020-12-27,8.670166e+05
727,2020-12-28,2.105212e+06
728,2020-12-29,2.110626e+06
729,2020-12-30,2.164505e+06
730,2020-12-31,2.179174e+06


In [ ]:
df_actions.head()

,AAPL,AXP,BA,BTC-USD,CAT,CSCO,CVX,DD,DIS,GS,HD,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PFE,PG,RTX,TRV,UNH,V,VZ,WBA,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-01-02,0,63,0,1,100,0,100,100,55,0,100,0,0,0,0,100,100,0,0,100,95,0,0,0,98,97,33,27,0,99,0
2019-01-03,74,100,0,-1,8,0,59,100,-55,0,-100,0,0,89,0,-100,100,0,0,66,14,5,0,82,100,-97,100,100,0,45,6
2019-01-04,100,100,0,100,100,0,100,100,100,0,0,14,0,-51,0,100,100,0,0,0,86,-5,0,100,100,57,100,-96,0,100,27
2019-01-05,0,0,0,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          3.080383e-01
Cumulative returns     1.179174e+00
Annual volatility      2.354038e+06
Sharpe ratio           1.482773e+00
Calmar ratio           3.080384e-01
Stability              2.921932e-04
Max drawdown          -9.999997e-01
Omega ratio            1.238884e+05
Sortino ratio          7.241984e+05
Skew                            NaN
Kurtosis                        NaN
Tail ratio             2.158230e+01
Daily value at risk   -2.827297e+05
dtype: float64


In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = '2019-01-01',
        end = '2021-01-01')

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (505, 8)
Annual return          0.144674
Cumulative returns     0.310981
Annual volatility      0.274619
Sharpe ratio           0.631418
Calmar ratio           0.390102
Stability              0.116677
Max drawdown          -0.370862
Omega ratio            1.149365
Sortino ratio          0.870084
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860710
Daily value at risk   -0.033911
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')

In [ ]:
print(df_actions.to_string())

            AAPL  AXP   BA  BTC-USD  CAT  CSCO   CVX    DD  DIS   GS   HD  IBM  INTC  JNJ   JPM    KO   MCD  MMM  MRK  MSFT  NKE  PFE   PG   RTX   TRV  UNH     V    VZ  WBA  WMT  XOM
date                                                                                                                                                                                  
2019-01-01     0    0    0        0    0     0     0     0    0    0    0    0     0    0     0     0     0    0    0     0    0    0    0     0     0    0     0     0    0    0    0
2019-01-02     0   63    0        1  100     0   100   100   55    0  100    0     0    0     0   100   100    0    0   100   95    0    0     0    98   97    33    27    0   99    0
2019-01-03    74  100    0       -1    8     0    59   100  -55    0 -100    0     0   89     0  -100   100    0    0    66   14    5    0    82   100  -97   100   100    0   45    6
2019-01-04   100  100    0      100  100     0   100   100  100    0    0   14     0 